In [74]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [71]:
overrides = f'--br_portfolio_path feb22 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
dispatch.dispatch_experiments('mar17', base_job_name='infocompare4', game_name='large_game_2', overrides = overrides, n_seeds=5)

# dispatch.dispatch_experiments('feb22', base_job_name='infocompare4', game_name='large_game_2', overrides = overrides, n_seeds=5)
# # dispatch.dispatch_experiments('feb22', base_job_name='infocompare3', game_name='large_game_2_hide', overrides = overrides, n_seeds=5)

# dispatch.dispatch_experiments('mar14', base_job_name='mar15_brier_lr', game_name='large_game_2', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('mar10', base_job_name='switch_and_brier3', game_name='large_game_2_switch10', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('feb22', base_job_name='switch_and_brier3_old_config', game_name='large_game_2_switch10', overrides = overrides, n_seeds=1)

# dispatch.dispatch_experiments('feb22', base_job_name='mar14_very_large_game_2', game_name='very_large_game_2', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('feb22', base_job_name='mar14_large_game_3', game_name='large_game_3', overrides = overrides, n_seeds=5)
# dispatch.dispatch_experiments('feb22', base_job_name='mar14_large_game_3', game_name='large_game_3_hide', overrides = overrides, n_seeds=5)
# dispatch.dispatch_experiments('feb22', base_job_name='tiny_game_Test', game_name='tiny', overrides = overrides, n_seeds=1)

Submitted batch job 10402
Submitted batch job 10403
Submitted batch job 10404
Submitted batch job 10405
Submitted batch job 10406


In [75]:
CONFIG = 'feb22'
GAME_NAME = 'large_game_2'
N_SEEDS = 5
overrides = f'--br_portfolio_path feb22 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'

# You have base and hidden demands inside of infocompare4 already
# Base
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

# Hidden demands
dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_hide', overrides = overrides, n_seeds=N_SEEDS)

# Reserve prices
# TODO: Holding off for now b/c rounds issue
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_low_reserve', overrides = overrides, n_seeds=N_SEEDS)
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_same_reserve', overrides = overrides, n_seeds=N_SEEDS)

# Activity
dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_activity', overrides = overrides, n_seeds=N_SEEDS)
# Undersell
dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_undersell', overrides = overrides, n_seeds=N_SEEDS)

# Initial pricing experiment, on the base game
dispatch.dispatch_experiments(CONFIG, base_job_name='pricing', game_name=f'{GAME_NAME}_pricing', overrides = overrides, n_seeds=N_SEEDS)


# TODO: Rerun all of these w/ pricing
# TODO: Rerun on 3p variants

Submitted batch job 10987
Submitted batch job 10988
Submitted batch job 10989
Submitted batch job 10990
Submitted batch job 10991
Submitted batch job 10992
Submitted batch job 10993
Submitted batch job 10994
Submitted batch job 10995
Submitted batch job 10996
Submitted batch job 10997
Submitted batch job 10998
Submitted batch job 10999
Submitted batch job 11000
Submitted batch job 11001
Submitted batch job 11002
Submitted batch job 11003
Submitted batch job 11004
Submitted batch job 11005
Submitted batch job 11006


In [27]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "_mlp  R" | awk '{print $1}' | xargs scancel

In [79]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R"

             JOBID                                                              NAME ST       TIME  NODES NODELIST(REASON)
             12201 br_1_mar19_large_game_2_low_reserve-feb22mlp-104_10000000_feb22_m  R    2:33:03      1 ip-10-255-7-35
             12206 eval_large_game_2_low_reserve-feb22mlp-104_10000000_mar19_feb22ml  R       4:26      1 ip-10-255-7-118


In [62]:
# !scancel 9587

In [69]:
!scancel -u ubuntu

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [37]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

Slurm accounting storage is disabled


In [50]:
from open_spiel.python.examples.ubc_plotting_utils import parse_run
from auctions.webutils import *

def find_best_checkpoint(run, max_t=None):
    ev_df = parse_run(run, max_t)
    best_t = ev_df.groupby('t')['ApproxNashConv'].first().idxmin()
    nash_conv_by_t = ev_df.groupby('t')['ApproxNashConv'].first()
    best_checkpoint = get_checkpoint(run, t=best_t)
    return nash_conv_by_t, best_checkpoint, nash_conv_by_t.min()


run = EquilibriumSolverRun.objects.last()
find_best_checkpoint(run)

(t
 1          215.790510
 1000000     17.236075
 2000000     14.384932
 3000000     19.374879
 4000000     15.771433
 5000000     13.261669
 6000000      9.768694
 Name: ApproxNashConv, dtype: float64,
 <EquilibriumSolverRunCheckpoint: large_game_2_switch-feb22mlp-100 Iteration 6000000>,
 9.768694201495364)

In [80]:
Experiment.objects.get(name='large_game_2_no_activity.json').delete()

DoesNotExist: Experiment matching query does not exist.

In [81]:
Experiment.objects.get(name='large_game_2_no_undersell.json').delete()

DoesNotExist: Experiment matching query does not exist.